<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/327_EaaS_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Utilities for EaaS Orchestrator Agent

In [ ]:
"""LLM Utilities for EaaS Orchestrator Agent

CEO-focused LLM enhancements for executive summaries and insights.
Incorporates principles from CEO Trust & Value Creation Guide.
"""

from typing import Dict, List, Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


def generate_executive_insights(
    summary: Dict[str, Any],
    agent_summaries: List[Dict[str, Any]],
    config: Any
) -> Optional[Dict[str, Any]]:
    """
    Generate CEO-friendly executive insights using LLM.

    Incorporates CEO Trust & Value Guide principles:
    - Business value first
    - Transparency and accountability
    - Actionable recommendations
    - CEO-friendly language

    Returns None if LLM fails (graceful degradation).
    """
    if not config.enable_llm_summaries:
        return None

    try:
        # Prepare data for LLM
        business_metrics = {
            "total_revenue_impact": summary.get('total_revenue_impact', 0.0),
            "total_cost": summary.get('total_cost', 0.0),
            "net_roi": summary.get('total_net_roi', 0.0),
            "roi_percent": summary.get('overall_roi_percent', 0.0),
            "cost_per_evaluation": summary.get('cost_per_evaluation', 0.0)
        }

        performance_metrics = {
            "overall_pass_rate": summary.get('overall_pass_rate', 0.0),
            "average_score": summary.get('average_score', 0.0),
            "healthy_agents": summary.get('healthy_agents', 0),
            "degraded_agents": summary.get('degraded_agents', 0),
            "critical_agents": summary.get('critical_agents', 0)
        }

        # Top performing agents
        top_agents = sorted(
            agent_summaries,
            key=lambda x: x.get('roi_percent', 0.0),
            reverse=True
        )[:3]

        # Agents needing attention
        agents_needing_attention = [
            a for a in agent_summaries
            if a.get('health_status') in ['degraded', 'critical'] or
               a.get('roi_percent', 0.0) < 50.0
        ]

        # Build prompt based on CEO Trust Guide principles
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an executive AI advisor specializing in AI agent evaluation and ROI analysis.
Your role is to provide clear, transparent, and actionable insights to CEOs and executives.

Core Principles (from CEO Trust & Value Creation Guide):
1. **Transparency First** - All insights must be explainable and data-driven
2. **Business Value Focus** - Every insight ties to revenue or cost
3. **CEO-Friendly Language** - Use business terms, avoid technical jargon
4. **Actionable Recommendations** - Every insight should lead to a clear action
5. **Statistical Rigor** - Acknowledge uncertainty, show confidence levels

Your communication style:
- Direct and concise
- Data-driven (cite specific numbers)
- Business-focused (revenue, cost, ROI)
- Action-oriented (what to do next)
- Transparent (acknowledge limitations)"""),

            ("human", """Analyze this AI agent evaluation report and provide executive insights.

**Business Metrics:**
- Total Revenue Impact: ${total_revenue_impact:,.2f}
- Total Cost: ${total_cost:,.2f}
- Net ROI: ${net_roi:,.2f} ({roi_percent:.1f}% return)
- Cost per Evaluation: ${cost_per_evaluation:.2f}

**Performance Metrics:**
- Overall Pass Rate: {overall_pass_rate:.1%}
- Average Score: {average_score:.2f}
- Healthy Agents: {healthy_agents}
- Degraded Agents: {degraded_agents}
- Critical Agents: {critical_agents}

**Top Performing Agents:**
{top_agents}

**Agents Needing Attention:**
{agents_needing_attention}

Provide:
1. **Executive Summary** (2-3 sentences): Key business value and ROI highlights
2. **Key Insights** (3-4 bullet points): Most important findings for decision-making
3. **Risk Assessment** (if any): Any concerns or areas needing attention
4. **Strategic Recommendations** (2-3 actionable items): What should the CEO do next?

Use CEO-friendly language. Focus on business value, ROI, and actionable insights.
Be transparent about any limitations or uncertainties in the data.""")
        ])

        # Format agent summaries for prompt
        top_agents_text = "\n".join([
            f"- {a.get('agent_id', 'unknown')}: ROI {a.get('roi_percent', 0.0):.1f}%, "
            f"Status: {a.get('health_status', 'unknown')}, "
            f"Cost: ${a.get('total_cost', 0.0):.2f}, "
            f"Revenue: ${a.get('revenue_impact', 0.0):.2f}"
            for a in top_agents
        ])

        agents_needing_attention_text = "\n".join([
            f"- {a.get('agent_id', 'unknown')}: ROI {a.get('roi_percent', 0.0):.1f}%, "
            f"Status: {a.get('health_status', 'unknown')}, "
            f"Score: {a.get('average_score', 0.0):.2f}"
            for a in agents_needing_attention
        ]) if agents_needing_attention else "None"

        # Initialize LLM
        llm = ChatOpenAI(
            model=config.llm_model,
            temperature=config.temperature,
            max_tokens=config.llm_summary_max_tokens
        )

        # Generate insights
        chain = prompt | llm
        response = chain.invoke({
            "total_revenue_impact": business_metrics["total_revenue_impact"],
            "total_cost": business_metrics["total_cost"],
            "net_roi": business_metrics["net_roi"],
            "roi_percent": business_metrics["roi_percent"],
            "cost_per_evaluation": business_metrics["cost_per_evaluation"],
            "overall_pass_rate": performance_metrics["overall_pass_rate"],
            "average_score": performance_metrics["average_score"],
            "healthy_agents": performance_metrics["healthy_agents"],
            "degraded_agents": performance_metrics["degraded_agents"],
            "critical_agents": performance_metrics["critical_agents"],
            "top_agents": top_agents_text,
            "agents_needing_attention": agents_needing_attention_text
        })

        # Parse response (expecting structured sections)
        insights_text = response.content.strip()

        # Extract sections (improved parsing)
        sections = {
            "executive_summary": "",
            "key_insights": [],
            "risk_assessment": "",
            "strategic_recommendations": []
        }

        # Split by numbered sections or headers
        lines = insights_text.split('\n')
        current_section = None
        current_content = []

        for line in lines:
            line_stripped = line.strip()
            if not line_stripped:
                continue

            line_lower = line_stripped.lower()

            # Detect section headers
            if any(phrase in line_lower for phrase in ['executive summary', 'summary:', 'overview:']):
                if current_section and current_content:
                    _save_section(sections, current_section, current_content)
                current_section = "executive_summary"
                current_content = []
            elif any(phrase in line_lower for phrase in ['key insight', 'insights:', 'takeaways:']):
                if current_section and current_content:
                    _save_section(sections, current_section, current_content)
                current_section = "key_insights"
                current_content = []
            elif any(phrase in line_lower for phrase in ['risk assessment', 'risks:', 'risk:']):
                if current_section and current_content:
                    _save_section(sections, current_section, current_content)
                current_section = "risk_assessment"
                current_content = []
            elif any(phrase in line_lower for phrase in ['recommendation', 'strategic recommendation', 'actions:', 'next steps:']):
                if current_section and current_content:
                    _save_section(sections, current_section, current_content)
                current_section = "strategic_recommendations"
                current_content = []
            else:
                # Skip markdown headers and limitations sections
                if line_stripped.startswith('#') or 'limitation' in line_lower:
                    continue
                # Clean up numbered lists (remove duplicate numbering)
                cleaned_line = _clean_numbered_line(line_stripped)
                # Also clean up double dashes from bullet points
                cleaned_line = cleaned_line.replace('- -', '-').replace('--', '-').strip()
                if cleaned_line:
                    current_content.append(cleaned_line)

        # Save last section
        if current_section and current_content:
            _save_section(sections, current_section, current_content)

        # Clean up recommendations - remove duplicates and format properly
        if sections["strategic_recommendations"]:
            sections["strategic_recommendations"] = _clean_recommendations(sections["strategic_recommendations"])

        # If parsing didn't work well, use full text as executive summary
        if not sections["executive_summary"]:
            sections["executive_summary"] = insights_text[:300] + "..." if len(insights_text) > 300 else insights_text

        return {
            "raw_response": insights_text,
            "executive_summary": sections["executive_summary"],
            "key_insights": sections["key_insights"] if isinstance(sections["key_insights"], list) else ([sections["key_insights"]] if sections["key_insights"] else []),
            "risk_assessment": sections["risk_assessment"],
            "strategic_recommendations": sections["strategic_recommendations"] if isinstance(sections["strategic_recommendations"], list) else ([sections["strategic_recommendations"]] if sections["strategic_recommendations"] else []),
            "source": "llm"
        }

    except Exception as e:
        print(f"    Warning: LLM insights generation failed: {str(e)}")
        print(f"    Falling back to rule-based summaries")
        return None


def _save_section(sections: Dict[str, Any], section_name: str, content: List[str]):
    """Helper to save section content properly."""
    if section_name in ["key_insights", "strategic_recommendations"]:
        sections[section_name] = content
    else:
        sections[section_name] = '\n'.join(content).strip()


def _clean_numbered_line(line: str) -> str:
    """Clean up numbered list items (remove duplicate numbering like '1. 1.')"""
    import re
    # Remove patterns like "1. 1." or "2. 2." etc.
    line = re.sub(r'^(\d+)\.\s*\1\.\s*', r'\1. ', line)
    # Remove standalone numbers at start
    line = re.sub(r'^(\d+)\.\s*$', '', line)
    return line.strip()


def _clean_recommendations(recommendations: List[str]) -> List[str]:
    """Clean and deduplicate recommendations."""
    cleaned = []
    seen = set()

    for rec in recommendations:
        rec_clean = rec.strip()
        if not rec_clean:
            continue

        # Remove duplicate numbering
        import re
        rec_clean = re.sub(r'^(\d+)\.\s*', '', rec_clean)  # Remove leading "1. ", "2. " etc.

        # Skip if it's a header or limitation
        if rec_clean.lower().startswith('#') or 'limitation' in rec_clean.lower():
            continue

        # Skip if it's a continuation sentence (starts with lowercase or "While", "However", etc.)
        # These are usually explanations, not actionable recommendations
        if rec_clean and rec_clean[0].islower() or rec_clean.startswith(('While ', 'However ', 'Although ', 'Since ', 'Given ')):
            continue

        # Deduplicate
        rec_normalized = rec_clean.lower().strip()
        if rec_normalized and rec_normalized not in seen:
            seen.add(rec_normalized)
            cleaned.append(rec_clean)

    return cleaned


def generate_agent_insights(
    agent_summary: Dict[str, Any],
    config: Any
) -> Optional[str]:
    """
    Generate CEO-friendly insights for a specific agent.

    Returns None if LLM fails (graceful degradation).
    """
    if not config.enable_llm_summaries:
        return None

    try:
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an executive AI advisor. Provide concise, business-focused insights about AI agent performance.
Focus on ROI, business value, and actionable recommendations. Use CEO-friendly language."""),

            ("human", """Provide a 2-3 sentence executive insight for this agent:

Agent: {agent_id}
ROI: {roi_percent:.1f}%
Status: {health_status}
Cost: ${total_cost:.2f}
Revenue Impact: ${revenue_impact:.2f}
Average Score: {average_score:.2f}
Total Evaluations: {total_evaluations}

Focus on business value and what this means for the organization.""")
        ])

        llm = ChatOpenAI(
            model=config.llm_model,
            temperature=config.temperature,
            max_tokens=150
        )

        chain = prompt | llm
        response = chain.invoke({
            "agent_id": agent_summary.get('agent_id', 'unknown'),
            "roi_percent": agent_summary.get('roi_percent', 0.0),
            "health_status": agent_summary.get('health_status', 'unknown'),
            "total_cost": agent_summary.get('total_cost', 0.0),
            "revenue_impact": agent_summary.get('revenue_impact', 0.0),
            "average_score": agent_summary.get('average_score', 0.0),
            "total_evaluations": agent_summary.get('total_evaluations', 0)
        })

        return response.content.strip()

    except Exception as e:
        print(f"    Warning: LLM agent insights failed: {str(e)}")
        return None



# Evaluation-as-a-Service Report

**Generated:** 2025-12-22 17:34:54

## Executive Summary

### 📊 Executive Overview

The AI agent evaluation demonstrates exceptional business value, with a total revenue impact of \$665 and an astonishing net ROI of 349,900%. All agents passed evaluation, indicating robust performance and no immediate concerns.

### 💰 Business Impact

- **Total Revenue Impact:** \$665.00
- **Total Cost:** \$0.19
- **Net ROI:** \$664.81 (349900.0% return)
- **Cost per Evaluation:** \$0.01
- **Agents with Positive ROI:** 4/4
- **Agents Needing Optimization:** 0

### 📊 Performance Metrics

- **Total Scenarios Evaluated:** 10
- **Total Evaluations:** 19
- **Overall Pass Rate:** 100.0% → Stable
- **Average Score:** 0.99 → Stable
- **Healthy Agents:** 4
- **Degraded Agents:** 0
- **Critical Agents:** 0

### 🎯 Key Takeaways

- - **Outstanding ROI**: The overall net ROI of \$664.81 reflects a highly efficient evaluation process, yielding a return of 349,900% on a minimal investment of \$0.19.
- - **Perfect Performance**: All agents achieved a 100% pass rate, with an average score of 0.99, indicating consistent high performance across the board.
- - **Top Performers**: The escalation agent, refund agent, and shipping update agent are driving significant revenue, with ROIs exceeding 249,000%, showcasing their effectiveness in generating business value.
- - **No Immediate Risks**: There are currently no agents requiring attention, suggesting a stable operational environment.

### ⚠️ Risk Assessment

There are no immediate risks or areas needing attention based on the current evaluation. However, continuous monitoring is recommended to maintain performance standards.


**Trends (vs. Previous Run):**
- **Stable:** 4 agent(s) with stable performance

### 🎯 Recommended Actions

1. **Leverage High-Performing Agents**: Consider scaling the use of the top-performing agents (escalation, refund, and shipping update) to maximize their revenue potential further.
2. **Invest in Continuous Improvement**: While current performance is excellent, invest in ongoing training and development for agents to sustain high performance and adapt to changing business needs.
3. **Monitor Performance Trends**: Establish a regular review process to track agent performance over time, ensuring any emerging issues are addressed promptly.
4. While the data shows strong performance, it is essential to remain vigilant for any changes in market conditions or agent performance that could impact future evaluations. Regular assessments will help mitigate potential risks.

## 💰 ROI & Business Value Analysis

### Overall Business Impact

- **Total Revenue Impact:** \$665.00
- **Total Cost:** \$0.19
- **Net ROI:** \$664.81
- **ROI Percentage:** 349900.0%
- **Cost per Evaluation:** \$0.01

### Agent-Level ROI

| Agent | Cost | Revenue Impact | Net ROI | ROI % | ROI Ratio | Trend | Status |
|-------|------|----------------|---------|-------|-----------|-------|--------|
| refund_agent | \$0.02 | \$100.00 | \$99.98 | 499899.8% | 5000.00x | → | ✅ Excellent |
| shipping_update_agent | \$0.07 | \$175.00 | \$174.93 | 249899.9% | 2500.00x | → | ✅ Excellent |
| apology_message_agent | \$0.06 | \$90.00 | \$89.94 | 149900.0% | 1500.00x | → | ✅ Excellent |
| escalation_agent | \$0.04 | \$300.00 | \$299.96 | 749899.8% | 7500.00x | → | ✅ Excellent |

## 📊 Agent Performance Details

### refund_agent

**Executive Insight:** The refund_agent demonstrates an exceptional ROI of 499,899.8%, indicating an extraordinary efficiency in processing refunds at a minimal cost of just \$0.02 per transaction. With a perfect average score of 1.00 from evaluations, this agent not only enhances customer satisfaction but also drives revenue impact, making it a critical asset for optimizing operational efficiency. To capitalize on this success, consider scaling its deployment across additional customer service functions to further amplify business value.

**Performance:**
- **Status:** healthy
- **Total Evaluations:** 2
- **Passed:** 2
- **Failed:** 0
- **Average Score:** 1.00 → Stable
- **Average Response Time:** 0.00s

**Business Value:**
- **Total Cost:** \$0.02
- **Revenue Impact:** \$100.00
- **Net ROI:** \$99.98 → Stable
- **ROI Percentage:** 499899.8% →
- **ROI Category:** exceptional
- **Cost per Evaluation:** \$0.01 → Stable

**Statistical Assessment:**
- *Historical comparison data not yet available*
- *KPI significance testing will be available after multiple evaluation runs*
- *ROI significance testing will be available after multiple evaluation runs*

### shipping_update_agent

**Executive Insight:** The shipping_update_agent demonstrates exceptional performance with an astonishing ROI of 249,899.9%, indicating that every dollar spent yields significant revenue impact. With a cost of just \$0.07 and an average score of 1.00 across evaluations, this agent is a highly efficient asset that can enhance customer satisfaction and operational efficiency. To maximize its value, consider scaling its deployment across additional customer touchpoints.

**Performance:**
- **Status:** healthy
- **Total Evaluations:** 7
- **Passed:** 7
- **Failed:** 0
- **Average Score:** 1.00 → Stable
- **Average Response Time:** 0.00s

**Business Value:**
- **Total Cost:** \$0.07
- **Revenue Impact:** \$175.00
- **Net ROI:** \$174.93 → Stable
- **ROI Percentage:** 249899.9% →
- **ROI Category:** exceptional
- **Cost per Evaluation:** \$0.01 → Stable

**Statistical Assessment:**
- *Historical comparison data not yet available*
- *KPI significance testing will be available after multiple evaluation runs*
- *ROI significance testing will be available after multiple evaluation runs*

### apology_message_agent

**Executive Insight:** The apology_message_agent demonstrates exceptional ROI at 149,900%, indicating a highly efficient use of resources with minimal cost of \$0.06 per interaction. With a strong average score of 0.97 from six evaluations, this agent not only enhances customer satisfaction but also drives a significant revenue impact of \$90.00, underscoring its value in maintaining brand reputation and customer loyalty. Prioritize scaling this agent to maximize its positive influence on customer relations and overall profitability.

**Performance:**
- **Status:** healthy
- **Total Evaluations:** 6
- **Passed:** 6
- **Failed:** 0
- **Average Score:** 0.97 → Stable
- **Average Response Time:** 0.00s

**Business Value:**
- **Total Cost:** \$0.06
- **Revenue Impact:** \$90.00
- **Net ROI:** \$89.94 → Stable
- **ROI Percentage:** 149900.0% →
- **ROI Category:** exceptional
- **Cost per Evaluation:** \$0.01 → Stable

**Statistical Assessment:**
**KPI Significance:**
- ➡️ Change not statistically significant: 0.0% change (p=1.0) | Target met: 0.97 >= 0.85
- **P-value:** 1.0000
- **Statistically Significant:** No
- **Percent Change:** 0.0%
- **95% Confidence Interval:** [0.850, 1.100]

**ROI Significance:**
- ✅ ROI \$89.94 is positive but not statistically significant | ROI Ratio: 1499.00x | 95% CI: \$89.94 to \$89.94
- **P-value:** nan
- **Statistically Significant:** No
- **ROI Ratio:** 1499.00x
- **95% Confidence Interval:** [89.94, 89.94]

### escalation_agent

**Executive Insight:** The escalation agent demonstrates exceptional performance with an ROI of 749,899.8%, indicating a highly efficient use of resources that translates into significant revenue impact for the organization. With a minimal cost of just \$0.04 per evaluation and a perfect average score, this agent is a prime candidate for scaling and further investment, ensuring continued enhancement of customer satisfaction and operational efficiency.

**Performance:**
- **Status:** healthy
- **Total Evaluations:** 4
- **Passed:** 4
- **Failed:** 0
- **Average Score:** 1.00 → Stable
- **Average Response Time:** 0.00s

**Business Value:**
- **Total Cost:** \$0.04
- **Revenue Impact:** \$300.00
- **Net ROI:** \$299.96 → Stable
- **ROI Percentage:** 749899.8% →
- **ROI Category:** exceptional
- **Cost per Evaluation:** \$0.01 → Stable

**Statistical Assessment:**
- *Historical comparison data not yet available*
- *KPI significance testing will be available after multiple evaluation runs*
- *ROI significance testing will be available after multiple evaluation runs*

## Evaluation Results

| Scenario | Agent | Score | Passed | Issues |
|----------|-------|-------|--------|--------|
| S001 | shipping_update_agent | 1.00 | ✓ |  |
| S002 | shipping_update_agent | 1.00 | ✓ |  |
| S002 | apology_message_agent | 1.00 | ✓ |  |
| S003 | refund_agent | 1.00 | ✓ |  |
| S003 | apology_message_agent | 0.85 | ✓ | Output status doesn't match expected outcome type |
| S004 | shipping_update_agent | 1.00 | ✓ |  |
| S004 | apology_message_agent | 1.00 | ✓ |  |
| S005 | escalation_agent | 1.00 | ✓ |  |
| S005 | apology_message_agent | 1.00 | ✓ |  |
| S006 | shipping_update_agent | 1.00 | ✓ |  |
| S006 | apology_message_agent | 1.00 | ✓ |  |
| S006 | escalation_agent | 1.00 | ✓ |  |
| S007 | shipping_update_agent | 1.00 | ✓ |  |
| S008 | shipping_update_agent | 1.00 | ✓ |  |
| S008 | apology_message_agent | 1.00 | ✓ |  |
| S008 | escalation_agent | 1.00 | ✓ |  |
| S009 | escalation_agent | 1.00 | ✓ |  |
| S009 | refund_agent | 1.00 | ✓ |  |
| S010 | shipping_update_agent | 1.00 | ✓ |  |
